In [2]:
import pandas as pd
import numpy as np
import nltk
import re


from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\srira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\srira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\srira\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
twitter=pd.read_csv("twitter.csv")
twitter.head()

twitter=twitter[["tweet","label"]]

In [5]:
twitter["label"].value_counts()

label
0    59440
1     4484
Name: count, dtype: int64

In [6]:
lemma=WordNetLemmatizer()
stop_words=set(stopwords.words("english"))

def clean_text(text):
  text=text.lower()
  text=re.sub(r"[^a-zA-Z0-9]"," ",text)
  words=nltk.word_tokenize(text) 
  return " ".join([lemma.lemmatize(word) for word in words if word not in stop_words])



twitter["tweet"]=twitter["tweet"].apply(lambda x:clean_text(x))
twitter.head()




,tweet,label
0,user father dysfunctional selfish drag kid dys...,0
1,user user thanks lyft credit use cause offer w...,0
2,bihday majesty,0
3,model love u take u time ur,0
4,factsguide society motivation,0


##word to vectors


-tokenization
-remove stopwords ,punctuation marks,specialcharecters
-stemming,lemmatization

techniques:
-bow(countVectorizer)
-tfidf
-word2vec

In [9]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score,confusion_matrix,classification_report



from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [10]:
x_train,x_test,y_train,y_test=train_test_split(twitter["tweet"],twitter["label"],test_size=0.3,random_state=42)
print(x_train.shape)
print(x_test.shape)

(44746,)
(19178,)


In [11]:
def training(x_train_vector,x_test_vector,y_train,y_test):

  models={"naivebayes":MultinomialNB(),
        "logisticregression":LogisticRegression(),
        "decisiontree":DecisionTreeClassifier(),
        "randomforest":RandomForestClassifier(),
        "knn":KNeighborsClassifier()
        }

  for name,model in models.items():
     
    model.fit(x_train_vector,y_train)
    y_test_pred = model.predict(x_test_vector)
    test_model_score = r2_score(y_test, y_test_pred)

    print(name+":")
    print("r2 score:",test_model_score)
    print(confusion_matrix(y_test,y_test_pred))
    print(classification_report(y_test,y_test_pred))
    print("-----------------------------------------------\n\n")

In [12]:
convert_vectors={"countVectorizer":CountVectorizer(max_features=500, binary=False ),"tfidf":TfidfVectorizer(max_features=100,ngram_range=(1,3))}



for name,vector in convert_vectors.items():
  print("vectorizer",name,":::")
  vectorizer=vector
  x_train_vector=vectorizer.fit_transform(x_train).toarray()
  x_test_vector=vectorizer.transform(x_test).toarray()
  training(x_train_vector,x_test_vector,y_train,y_test)

vectorizer countVectorizer :::
naivebayes:
r2 score: 0.0070619012995203345
[[17288   519]
 [  745   626]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     17807
           1       0.55      0.46      0.50      1371

    accuracy                           0.93     19178
   macro avg       0.75      0.71      0.73     19178
weighted avg       0.93      0.93      0.93     19178

-----------------------------------------------


logisticregression:
r2 score: 0.19402334709913605
[[17694   113]
 [  913   458]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     17807
           1       0.80      0.33      0.47      1371

    accuracy                           0.95     19178
   macro avg       0.88      0.66      0.72     19178
weighted avg       0.94      0.95      0.94     19178

-----------------------------------------------


decisiontree:
r2 score: 0.4477567378271857
[[17509   298]
 

In [ ]:
!pip install lightgbm

In [16]:

import lightgbm as lgb

vectorizer=TfidfVectorizer(max_features=700, ngram_range=(1,3) )

train_data = lgb.Dataset(vectorizer.fit_transform(x_train).toarray(), label=y_train)
test_data = lgb.Dataset(vectorizer.transform(x_test).toarray(), label=y_test, reference=train_data)

In [22]:

params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, test_data],
    num_boost_round=1000,
)
y_pred = model.predict(vectorizer.transform(x_test).toarray())

y_pred=[1 if i>0.5 else 0 for i in y_pred]


[LightGBM] [Info] Number of positive: 3113, number of negative: 41633
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059880 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42875
[LightGBM] [Info] Number of data points in the train set: 44746, number of used features: 700
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.069570 -> initscore=-2.593306
[LightGBM] [Info] Start training from score -2.593306


In [23]:
test_model_score = r2_score(y_test, y_pred)

print("r2 score:",test_model_score)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("-----------------------------------------------\n\n")

r2 score: 0.4681810974523538
[[17706   101]
 [  576   795]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     17807
           1       0.89      0.58      0.70      1371

    accuracy                           0.96     19178
   macro avg       0.93      0.79      0.84     19178
weighted avg       0.96      0.96      0.96     19178

-----------------------------------------------




In [25]:
import pickle
import joblib
pickle.dump(vectorizer,open("vectorizer.pkl","wb"))


In [26]:
model.save_model("lightgbm.txt")
lightgbm_model=lgb.Booster(model_file="lightgbm.txt")


pickle.dump(lightgbm_model,open("light_gbm_model.pkl","wb"))


In [28]:
#testing pkl files


vectormodel=pickle.load(open("vectorizer.pkl","rb"))

gblmodel=pickle.load(open("light_gbm_model.pkl","rb"))


In [30]:


ypred=gblmodel.predict(vectormodel.transform(x_test).toarray())

ypred=[1 if i>0.5 else 0 for i in y_pred]
test_model_score = r2_score(y_test, ypred)

print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))
print("-----------------------------------------------\n\n")

[[17706   101]
 [  576   795]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     17807
           1       0.89      0.58      0.70      1371

    accuracy                           0.96     19178
   macro avg       0.93      0.79      0.84     19178
weighted avg       0.96      0.96      0.96     19178

-----------------------------------------------


